## This code apply Pre_trained Resnet50 Model on below Stacked dataset:
1. Stack [Originial + Histogram_Median ]
2. Stack [Originial + Sharpened Images ]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load originial Dataset

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/UB_Study/CVIP_Project/FERDataset.zip"
extract_to = "/content/FERDataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Dataset unzipped to local Colab storage.")

Dataset unzipped to local Colab storage.


In [ ]:
import os
dir = "/content/FERDataset"
def count_files(directory):
    total_files = 0
    no_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
        no_files += len(files)
        print(root,dirs,no_files)
        no_files=0
    return total_files

print("Total files extracted:", count_files(dir))

/content/FERDataset ['test', 'train'] 0
/content/FERDataset/test ['angry', 'disgust', 'surprise', 'happy', 'fear', 'neutral', 'sad'] 0
/content/FERDataset/test/angry [] 958
/content/FERDataset/test/disgust [] 111
/content/FERDataset/test/surprise [] 831
/content/FERDataset/test/happy [] 1774
/content/FERDataset/test/fear [] 1024
/content/FERDataset/test/neutral [] 1233
/content/FERDataset/test/sad [] 1247
/content/FERDataset/train ['angry', 'disgust', 'surprise', 'happy', 'fear', 'neutral', 'sad'] 0
/content/FERDataset/train/angry [] 3995
/content/FERDataset/train/disgust [] 436
/content/FERDataset/train/surprise [] 3171
/content/FERDataset/train/happy [] 7215
/content/FERDataset/train/fear [] 4097
/content/FERDataset/train/neutral [] 4965
/content/FERDataset/train/sad [] 4830
Total files extracted: 35887


## Load Histogram_Median Dataset

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/UB_Study/CVIP_Project/Preprocessed_Dataset/Histogram_Median.zip"
extract_to = "/content/Histogram_Median"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Dataset unzipped to local Colab storage.")

Dataset unzipped to local Colab storage.


In [ ]:
import os
dir = "/content/Histogram_Median/Histogram_Median"
def count_files(directory):
    total_files = 0
    no_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
        no_files += len(files)
        print(root,dirs,no_files)
        no_files=0
    return total_files

print("Total files extracted:", count_files(dir))

/content/Histogram_Median/Histogram_Median ['test', 'train'] 0
/content/Histogram_Median/Histogram_Median/test ['angry', 'disgust', 'surprise', 'happy', 'fear', 'neutral', 'sad'] 0
/content/Histogram_Median/Histogram_Median/test/angry [] 958
/content/Histogram_Median/Histogram_Median/test/disgust [] 111
/content/Histogram_Median/Histogram_Median/test/surprise [] 831
/content/Histogram_Median/Histogram_Median/test/happy [] 1774
/content/Histogram_Median/Histogram_Median/test/fear [] 1024
/content/Histogram_Median/Histogram_Median/test/neutral [] 1233
/content/Histogram_Median/Histogram_Median/test/sad [] 1247
/content/Histogram_Median/Histogram_Median/train ['angry', 'disgust', 'surprise', 'happy', 'fear', 'neutral', 'sad'] 0
/content/Histogram_Median/Histogram_Median/train/angry [] 3995
/content/Histogram_Median/Histogram_Median/train/disgust [] 436
/content/Histogram_Median/Histogram_Median/train/surprise [] 3171
/content/Histogram_Median/Histogram_Median/train/happy [] 7215
/content/

## Implementation of Stacking the Images

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os
import numpy as np

class StackedImageDataset(Dataset):
    def __init__(self, orig_root_dir, sharp_root_dir, transform=None):
        self.orig_root_dir = orig_root_dir
        self.sharp_root_dir = sharp_root_dir
        self.transform = transform
        self.samples = []

        for class_name in os.listdir(orig_root_dir):
            orig_class_dir = os.path.join(orig_root_dir, class_name)
            sharp_class_dir = os.path.join(sharp_root_dir, class_name)

            for fname in os.listdir(orig_class_dir):
                orig_path = os.path.join(orig_class_dir, fname)
                sharp_path = os.path.join(sharp_class_dir, fname)
                if os.path.exists(sharp_path):
                    self.samples.append((orig_path, sharp_path, class_name))

        self.class_to_idx = {name: idx for idx, name in enumerate(sorted(os.listdir(orig_root_dir)))}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        orig_path, sharp_path, label = self.samples[idx]

        orig_img = Image.open(orig_path).convert('L')
        sharp_img = Image.open(sharp_path).convert('L')

        orig = np.array(orig_img)
        sharp = np.array(sharp_img)
        stacked = np.stack([orig, sharp, (sharp - orig + 128)], axis=-1)
        stacked = np.clip(stacked, 0, 255).astype(np.uint8)
        stacked_img = Image.fromarray(stacked)

        if self.transform:
            stacked_img = self.transform(stacked_img)

        return stacked_img, self.class_to_idx[label]

def train():
  for epoch in range(num_epochs):
    model.train()
    correct_train, total_train, train_loss = 0, 0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
        train_loss += loss.item() * images.size(0)

    train_acc = correct_train / total_train * 100
    print(f"Epoch {epoch+1}: Train Acc: {train_acc:.2f}%")


def evaluate ():
    model.eval()
    correct_test, total_test, test_loss = 0, 0, 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)
            test_loss += loss.item() * images.size(0)


    test_acc = correct_test / total_test * 100
    print(f"\n Test Accuracy: {test_acc:.2f}%")


## Applying Resnet50 model on Stack [Originial + Histogram_Median ]

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

train_dataset = StackedImageDataset(
    orig_root_dir='/content/FERDataset/train',
    sharp_root_dir='/content/Histogram_Median/Histogram_Median/train',
    transform=transform
)

test_dataset = StackedImageDataset(
    orig_root_dir='/content/FERDataset/test',
    sharp_root_dir='/content/Histogram_Median/Histogram_Median/test',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 7)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 15

train_acc_list, test_acc_list, train_loss_list, test_loss_list = [], [], [], []


In [ ]:
train()

Epoch 1: Train Acc: 52.09%
Epoch 2: Train Acc: 67.18%
Epoch 3: Train Acc: 78.02%
Epoch 4: Train Acc: 88.91%
Epoch 5: Train Acc: 94.27%
Epoch 6: Train Acc: 95.58%
Epoch 7: Train Acc: 96.43%
Epoch 8: Train Acc: 96.70%
Epoch 9: Train Acc: 96.58%
Epoch 10: Train Acc: 97.38%
Epoch 11: Train Acc: 97.28%
Epoch 12: Train Acc: 97.58%
Epoch 13: Train Acc: 97.52%
Epoch 14: Train Acc: 97.59%
Epoch 15: Train Acc: 97.93%


In [ ]:
evaluate()


 Test Accuracy: 61.70%


In [ ]:
torch.save(model.state_dict(), 'Resnet50_Stack_Ori_Hist_Median.pth')

#### Resnet Model on Stacking of original and Histogram_median dataset gave the accuracy of 61.70%

## Applying Resnet50 on Stack [Originial + Sharpened Images ]

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/UB_Study/CVIP_Project/Preprocessed_Dataset/Sharpened.zip"
extract_to = "/content/Sharpened"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Dataset unzipped to local Colab storage.")

Dataset unzipped to local Colab storage.


In [ ]:
import os
dir = "/content/Sharpened/Sharpened"
def count_files(directory):
    total_files = 0
    no_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
        no_files += len(files)
        print(root,dirs,no_files)
        no_files=0
    return total_files

print("Total files extracted:", count_files(dir))

/content/Sharpened/Sharpened ['test', 'train'] 0
/content/Sharpened/Sharpened/test ['angry', 'disgust', 'surprise', 'happy', 'fear', 'neutral', 'sad'] 0
/content/Sharpened/Sharpened/test/angry [] 958
/content/Sharpened/Sharpened/test/disgust [] 111
/content/Sharpened/Sharpened/test/surprise [] 831
/content/Sharpened/Sharpened/test/happy [] 1774
/content/Sharpened/Sharpened/test/fear [] 1024
/content/Sharpened/Sharpened/test/neutral [] 1233
/content/Sharpened/Sharpened/test/sad [] 1247
/content/Sharpened/Sharpened/train ['angry', 'disgust', 'surprise', 'happy', 'fear', 'neutral', 'sad'] 0
/content/Sharpened/Sharpened/train/angry [] 3995
/content/Sharpened/Sharpened/train/disgust [] 436
/content/Sharpened/Sharpened/train/surprise [] 3171
/content/Sharpened/Sharpened/train/happy [] 7215
/content/Sharpened/Sharpened/train/fear [] 4097
/content/Sharpened/Sharpened/train/neutral [] 4965
/content/Sharpened/Sharpened/train/sad [] 4830
Total files extracted: 35887


In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

train_dataset = StackedImageDataset(
    orig_root_dir='/content/FERDataset/train',
    sharp_root_dir='/content/Sharpened/Sharpened/train',
    transform=transform
)

test_dataset = StackedImageDataset(
    orig_root_dir='/content/FERDataset/test',
    sharp_root_dir='/content/Sharpened/Sharpened/test',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 7)  # 7 emotions in FER
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 15

train()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 194MB/s]


Epoch 1: Train Acc: 57.93%
Epoch 2: Train Acc: 71.31%
Epoch 3: Train Acc: 81.21%
Epoch 4: Train Acc: 89.75%
Epoch 5: Train Acc: 94.22%
Epoch 6: Train Acc: 95.74%
Epoch 7: Train Acc: 96.45%
Epoch 8: Train Acc: 97.32%
Epoch 9: Train Acc: 96.96%
Epoch 10: Train Acc: 96.93%
Epoch 11: Train Acc: 97.49%
Epoch 12: Train Acc: 97.82%
Epoch 13: Train Acc: 97.64%
Epoch 14: Train Acc: 97.84%
Epoch 15: Train Acc: 98.01%


In [ ]:
evaluate()


 Test Accuracy: 64.64%


In [ ]:
torch.save(model.state_dict(), 'Resnet50_Stack_Ori_Sharpen.pth')

### Resnet50 model on Original + Sharpened dataset gave the highest accuracy among the filtered stacked images.